# Measurement uncertainty calculations

# Introduction

### Definitions
**Measurement uncertainty** is defined in the ISO ‘Guide to the Expression of Uncertainty in Measurement’ as ‘a parameter, associated with the result of a measurement that characterises the dispersion of the values that  could  reasonably  be  attributed  to  the  measurand’.  The  **measurand**  is  the  ‘quantity  intended  to  be measured’.

### Purpose
An estimate  of  uncertainty  provides  a  quantitative  indication  of  the  quality  of  a  measurement  result. Rather than using a ‘bottom-up’ approach which examines the inputs to a method and considers how they might influence results, the biochemical genetics units uses a ‘top-down’ approach using the information from method outputs (e.g. the observed variability of replicate measurement results), as described in ADD.BIO 6680: Estimating measurement uncertainty in the biochemical genetics unit. This notebook attempts to outline these steps.

## Imports, configuration

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Optional Pandas configuration.

Since some columns have long names, change the maximum column width in Pandas so we can see the full name. We also may want to see the full list of rows, in some cases.


In [ ]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 50)

## Import mappings

The mappings file contains:
* a sort order for measurands within each assay
* a flag if the measurand is a ratio
* mappings to link the Assay and Measurand to the name of the QC material in [Randox Acusera 24:7](https://qc.randox.com/Acusera#Home)
* mappings to link the EQA scheme and EQA analyte name
* the lower and upper limit of quantitation, as described in the assay SOP. If the lower or upper limits are not specified in the SOP, these are given as 0 and 9999999 respectively. If more than one limit is provided (for example, different limits for different instruments), then the lower lower limit or higher upper limit was used.


In [ ]:
mappings = pd.read_csv('data\\raw_data\\mappings.csv')
mappings.head()

# Obtain an estimate of the precision of the measurement procedure

## Load measurement uncertainty report from Randox Acusera 24:7

The measurement uncertainty report contains stiastics on each lot number in Randox Acusera. Data points that have been rejected are not included in the analysis.

When loading the data, merge with the mappings table and exclude QC lots where the measurand concentration is outside the reportable range

In [ ]:
from functions import load_randox

In [ ]:
qc_data = load_randox("data\\raw_data\\uncertainty_of_measurement_010120_311220.xlsx"
                        ,"data\\raw_data\\mappings.csv")

## Obtain an estimate of the precision of the measurement procedure

The measurement uncertainty report from Randox needs to be processed to:
* exclude QC lots where the measurand concentration is outside the reportable range (done above when importing the data)
* exclude QC lots with low counts
* calculate averages for each measurand
* present the data by assay in a useful format

First,  create a summary for each lot number that shows some basic statistics by pivoting existing data and removing lots with low counts.

In [ ]:
from functions import qc_lot_summary

In [ ]:
qc_lot_summary(qc_data,'Organic acids', 10)

In [ ]:
qc_lot_summary(qc_data,'Newborn screening for inherited metabolic disorders',10)

In [ ]:
qc_lot_summary(qc_data,'Acylcarnitines (Blood spot)',10)

Next, we aggregate the data over all lots numbers and instruments (excluding those whe the count is less than the count threshold) by summing the total counts and averaging the measurement uncertainty and %CV.

In [ ]:
from functions import qc_aggregated

In [ ]:
def qc_aggregated(qc_data, assay, count_thresh=1):
    '''
    Returns total number of QC datapoints for each analyte and average of each lot number measurement uncertainty and %CV
    Excludes any statistics with fewer than count_thresh values
    '''
    mappings = pd.read_csv("data\\raw_data\\mappings.csv")
    all_measurands = mappings[mappings['Assay'] == assay]
    all_measurands = all_measurands.reset_index()[['Measurand','Order']]
    
    ## Filter qc data for assay values only
    filtered = qc_data[qc_data['Assay'] == assay].drop(columns=['Assay','Randox'])
    
    ## Ignore lots with low numbers of data points
    filtered = filtered[filtered['Count'] >= count_thresh]
    
    ## Calculate aggregate functions
    aggregated = filtered.groupby('Measurand').agg({'Count':'sum',
                                              'UOM':'mean',
                                              '% CV':'mean'})
    
    ## Merge with mappings to show all measurands (so that they appear even if no QC data) and order by sort order
    aggregated = aggregated.merge(right=all_measurands,on='Measurand',how='outer')
    aggregated = aggregated.sort_values(by='Order', ignore_index=True)
    aggregated = aggregated.drop(columns='Order')
    aggregated = aggregated.set_index('Measurand')
    
    ## Fill blanks
    aggregated = aggregated.fillna('')
    
    return aggregated.round(2)

In [ ]:
qc_aggregated(qc_data,'Organic acids',10)

In [ ]:
qc_aggregated(qc_data,'Acylcarnitines (Blood spot)',10)

We can then combine the lot statistics and aggregated statistics

In [ ]:
from functions import qc_lot_summary_with_means

Note: for the acylcarnitines assay, some measurands have data for one QC lot, and some measurands have no values. This is because the measurand is present at less than the limit of quantitation

In [ ]:
def qc_lot_summary_with_means(qc_data, assay, count_thresh=1):
    '''
    Returns a dataframe containing both lot number statistics and aggregated statistics
    Excludes any statistics with fewer than count_thresh values
    '''
    assay_qc_pivot = qc_lot_summary(qc_data, assay, count_thresh)
    aggregated = qc_aggregated(qc_data, assay, count_thresh)
    
    assay_qc_pivot[('All instrument','All lots','Count')] = aggregated['Count']
    assay_qc_pivot[('All instrument','All lots','UOM')] = aggregated['UOM']
    assay_qc_pivot[('All instrument','All lots','% CV')] = aggregated['% CV']
    
    ## Fill blanks
    assay_qc_pivot = assay_qc_pivot.fillna('')
    
    return assay_qc_pivot.round(2)

In [ ]:
qc_lot_summary_with_means(qc_data, 'Newborn screening for inherited metabolic disorders', 10)

We can export the data for all assays as seperate .csv files which are saved in the \data\processed\qc_summary_tables folder

In [ ]:
from functions import assay_qc_data_export

In [ ]:
assay_qc_data_export(qc_data,10)

# Obtain an estimate of the measurement bias and its uncertainty

Measurement bias and its uncertainty can be estimnated from the regular participation in external quality assessment (EQA).

## Load and pre-processing EQA data

### Load EQA data

Load EQA results from a csv file (UKNEQAS results can be obtained using EQA data scraper)

In [ ]:
from functions import load_eqa

In [ ]:
folder = 'data//raw_data//eqa_results//results_targets'
eqa_results = load_eqa(folder)
eqa_results.head()

### Calculate EQA statistics

The EQA data does not contain the specimen bias or percentage uncertainty in the target. We can calculate these from the result, target value and standard uncertainty of the target value (after excluding non-numeric results and target values)

In [ ]:
from functions import eqa_calculations

In [ ]:
eqa_calculated = eqa_calculations(eqa_results)

### Exclude outliers

Let's have a look at any outliers, where the bias is 100% or more.

In [ ]:
eqa_calculated[abs(eqa_calculated['% Bias']) >= 100]

Exclude these outliers

In [ ]:
eqa_calculated = eqa_calculated[abs(eqa_calculated['% Bias']) < 100]

### Merge data with assay names and measurands

Merge the data with the mappings file, since the EQA schemes don't correspond with the assay names.

In [ ]:
eqa_data = mappings.merge(right=eqa_calculated,on=['Scheme name','EQA analyte name'],how='outer')

## Plot bias against target value

Next plot the data to see if bias varies with target value. This can be done either individually, or for all assays in the EQA data.

First plot for a single EQA scheme:

In [ ]:
from functions import eqa_assay_bias_plot

In [ ]:
#eqa_assay_bias_plot(eqa_data,'Amino acids (Plasma)')

Next, plot for all assays in the eqa data

In [ ]:
from functions import eqa_bias_multi_plot

In [ ]:
#eqa_bias_multi_plot(eqa_data)

## Calculate EQA summary statistics

Now we can calculate aggregate statistics across all specimens and distributions. This assumes that the %bias is constant throughout the measurable range (may not be true, although the previous plots will give an indication of how valid this assumption is).

In [ ]:
from functions import eqa_summary_statistics

In [ ]:
eqa_data.head()

In [ ]:
def eqa_summary_statistics(df,assay):
    '''
    Display a table showing the mean bias, standard deviation of the bias, average % uncertainty in the target value,
    combined uncertainty of the mean % bias and expanded uncertainty of % bias (using a coverage factor of 2)
    for the specified assay
    '''
    eqa_summary = df[df['Assay'] == assay]
    eqa_summary = eqa_summary.groupby('Measurand').agg({'Specimen':'count','Targ':['min','max'],'% Bias':['mean','std'],'% uncertainty in target value':'mean'})
    
    ## Function used to combined uncertainties
    def combined_uncertainty(std,target_value_uncert):
        return np.sqrt(std**2 + target_value_uncert**2)
    
    ## Combine standard deviation of bias with average uncertainty in the target value
    eqa_summary['Combined uncertainty of % bias'] = eqa_summary.apply(lambda x: combined_uncertainty(x[('% Bias','std')],x[('% uncertainty in target value', 'mean')]), axis = 1)
    
    ## Calculate expanded uncertainty in the bias estimate (using coverage factor k=2)
    eqa_summary['Expanded uncertainty of % bias'] = eqa_summary['Combined uncertainty of % bias'].apply(lambda x: x*2)

    ## Merge with all measurands and sort
    mappings = pd.read_csv("data\\raw_data\\mappings.csv")
    all_measurands = mappings[mappings['Assay'] == assay]
    all_measurands = all_measurands.reset_index()[['Measurand','Order']]
    
    eqa_summary = eqa_summary.merge(right=all_measurands,on='Measurand',how='outer')
    eqa_summary = eqa_summary.sort_values(by='Order', ignore_index=True)
    eqa_summary = eqa_summary.drop(columns='Order')   

    ## some jiggery pokery to get the multindex back
    eqa_summary = eqa_summary.set_index('Measurand')
    eqa_summary.columns = pd.MultiIndex.from_tuples(eqa_summary.columns)
    
    ## Fill blanks
    eqa_summary = eqa_summary.fillna('')
    
    return round(eqa_summary,1)


In [ ]:
eqa_stats = eqa_summary_statistics(eqa_data,'Newborn screening for inherited metabolic disorders')
eqa_stats

If there is no quantitative EQA scheme (like for acylcarnitines), the table will be blank

In [ ]:
eqa_stats = eqa_summary_statistics(eqa_data,'Acylcarnitines (Blood spot)')
eqa_stats

We can export the data for all assays as seperate .csv files which are saved in the \data\processed\eqa_summary_tables folder

In [ ]:
from functions import assay_eqa_data_export

In [ ]:
assay_eqa_data_export(eqa_data)

# Performance targets

Performance targets for each measurand are detailed in **ADD.BIO 6378**: *BGU Quality Monitoring Processes (Appendix 4)*

Import the performance targets, rename the columns and round to one decimal place.

In [ ]:
from functions import load_performance_targets

In [ ]:
performance_targets = load_performance_targets('data//raw_data/performance_targets_january_2021.xlsx')
performance_targets.head()

Performance against imprecision, bias and total allowable error is assessed against three targets:
* Optimal
* Desirable
* Minimal

In [ ]:
from functions import performance_table

In [ ]:
def performance_table(qc_data, eqa_data, assay, count_thresh, targets):
    '''
    Create table summarising imprecision, bias and total error performance against
    performance targets
    '''
    ### Load imprecision data
    imprec = qc_aggregated(qc_data,assay,count_thresh)
    
    # Select only the % CV column and measurand name
    imprec = imprec.reset_index()[['Measurand','% CV']]
    
    ### Load bias data
    bias = eqa_summary_statistics(eqa_data,assay)
    
    # Select only the % Bias column and measurand name
    bias.columns = bias.columns.get_level_values(0)
    bias = bias.reset_index()[['Measurand','Combined uncertainty of % bias']]
    bias = bias.rename(columns={'Combined uncertainty of % bias':'% Bias'})
    
    ### Merge imprecision and bias data
    df = imprec.merge(right=bias,on='Measurand',how='outer')
    df[['% CV','% Bias']] = df[['% CV','% Bias']].apply(pd.to_numeric, errors='ignore')
    
    ### Calculate total error
    def total_error(cv,bias):
        return round(abs(bias) + 1.65*cv,1)
    
    df['Total error'] = df.apply(lambda x: total_error(x['% CV'],x['% Bias']),axis=1)
    
    ### Merge with performance targets
    targets = targets[targets['Assay'] == assay]
    df = df.merge(right=targets,on='Measurand',how='outer')
    #df = df.fillna('')
    
    ### Calculate performance against performance targets
    def performance(value, optimal, desirable, minimal):
        if value < optimal:
            return "Optimal"
        elif value < desirable:
            return "Desirable"
        elif value < minimal:
            return "Minimal"
        else:
            return "Not met"
    
    df['CV performance'] = df.apply(lambda x: performance(x['% CV'],x['Anal CV Optimal'],x['Anal CV Desirable'],x['Anal CV Minimal']), axis = 1)
    df['Bias performance'] = df.apply(lambda x: performance(x['% Bias'],x['Bias Optimal'],x['Bias Desirable'],x['Bias Minimal']), axis = 1)
    df['TAE performance'] = df.apply(lambda x: performance(x['Total error'],x['TE Optimal'],x['TE Desirable'],x['TE Minimal']), axis = 1)
    
    ## Select only required columns
    df = df[['Measurand','% CV','CV performance','% Bias','Bias performance','Total error','TAE performance']]
    df = df.set_index('Measurand')
    df[['% CV','% Bias','Total error']] = df[['% CV','% Bias','Total error']].fillna('')
    
    return df

In [ ]:
performance_table(qc_data, eqa_data, 'Acylcarnitines (Blood spot)', 10, performance_targets)

In [ ]:
performance_table(qc_data, eqa_data, 'Amino acids (Plasma)', 10, performance_targets)

In [ ]:
def assay_performance_data_export(qc_data, eqa_data, count_thresh, targets):
    '''
    Createsa a performance summary table for each assay.
    
    Exports each table to a .csv file in the processed data folder
    '''
    for assay in qc_data['Assay'].unique():
        try:
            filepath = os.path.abspath('') + '\\data\\processed\\performance_summary_tables\\' + assay + '.csv'
            table =  performance_table(qc_data, eqa_data, assay, count_thresh, targets)
            table.to_csv(filepath)
            print(f'{assay} succesfully exported')
        except:
            print(f'!!! Error in exporting data for {assay}')

In [ ]:
assay_performance_data_export(qc_data, eqa_data, 10, performance_targets)